In [55]:
import pandas as pd
import numpy as np
from datetime import *
from dateutil.relativedelta import relativedelta
from xlrd import *
import warnings
warnings.filterwarnings("ignore")
from functools import reduce
from tqdm.notebook import tqdm
import utils as ut
from scipy import linalg


In [56]:
## 读取数据
# fqclose = pd.read_csv("stylefactor/fqclose.csv", index_col=0)
# ret5 = fqclose.pct_change(5, fill_method=None)

# FactorRtn = pd.read_csv('factor_return5.csv', index_col=0)

# SpecificRtn = ret5
# MarketValue = pd.read_csv("stylefactor/floatmktcap.csv", index_col=0)
FactorRtn = pd.read_pickle('BarraFactorData/FactorReturn.pkl')
SpecificRtn = pd.read_pickle('BarraFactorData/SpecificReturn.pkl').dropna(how='all',axis=1).iloc[:, :50]
MarketValue = pd.read_csv('stylefactor/floatmktcap.csv', index_col=0).iloc[:, :50]

In [3]:
# 只需要跑一次，保留因子暴露，每天暴露存一个文件
beta = pd.read_csv("BarraFactorData/Beta.csv", index_col=0).iloc[:, :50]
bp = pd.read_csv("BarraFactorData/BookToPrice.csv", index_col=0).iloc[:, :50]
ey = pd.read_csv("BarraFactorData/EarningsYield.csv", index_col=0).iloc[:, :50]
growth = pd.read_csv("BarraFactorData/Growth.csv", index_col=0).iloc[:, :50]
leverage = pd.read_csv("BarraFactorData/Leverage.csv", index_col=0).iloc[:, :50]
liq = pd.read_csv("BarraFactorData/Liquidity.csv", index_col=0).iloc[:, :50]
mom = pd.read_csv("BarraFactorData/Momentum.csv", index_col=0).iloc[:, :50]
nlsize = pd.read_csv("BarraFactorData/NonlinearSize.csv", index_col=0).iloc[:, :50]
resvol = pd.read_csv("BarraFactorData/ResidualVolatility.csv", index_col=0).iloc[:, :50]
size = pd.read_csv("BarraFactorData/Size.csv", index_col=0).iloc[:, :50]
indus = pd.read_pickle("BarraFactorData/Industry.pkl")

# for i in range(len(beta)):
#     print(i, beta.iloc[i].name)
#     indus_dummies = pd.get_dummies(indus.iloc[i])
#     if 'Real Estate' not in indus_dummies:
#         print("刚开始没有房地产这个行业，加一列0")
#         indus_dummies['Real Estate'] = 0

#     date_df = pd.concat([beta.iloc[i], bp.iloc[i],ey.iloc[i], growth.iloc[i],
#                          leverage.iloc[i], liq.iloc[i], mom.iloc[i],
#                          nlsize.iloc[i], resvol.iloc[i], size.iloc[i],
#                          # 注意这里有个坑，行业刚开始是31个后面变成了32个。需要调整
#                          indus_dummies], axis=1)
#     date_df.columns = ['Beta', 'BookToPrice', 'EarningsYield', 'Growth', 'Leverage',
#                        'Liquidity', 'Momentum', 'NonlinearSize', 'ResidualVolatility', 'Size'] + indus_dummies.columns.tolist()
#     date_df.insert(0, 'Market', 1)
#     date_df = date_df[FactorRtn.columns]
#     date_df.to_pickle("fac_data/fac_data" + str(beta.iloc[i].name) + ".pkl")

In [4]:
indus = pd.read_pickle("BarraFactorData/Industry.pkl").iloc[:, :50]

数据预处理 参数设定

In [134]:
MarketValue.index = MarketValue.index.astype(str)
FactorsValid = FactorRtn.columns.to_list()
TradeDates = FactorRtn.index.intersection(SpecificRtn.index).to_list()

FactorRtn = FactorRtn.loc[TradeDates,:] # (T, K)
SpecificRtn = SpecificRtn.loc[TradeDates,:] # (T, N)
MarketValue = MarketValue.loc[TradeDates,:] # (T, N)

T = FactorRtn.shape[0] # 历史样本数
K = FactorRtn.shape[1] # 风险因子数

## 设定计算参数
K_inds = 32
length = 243
lag_F_Corr = 2
tau_F_Corr = 480
lag_F_Vol = 5
# 周度调整为20，月度为81和42
tau_F_Vol = 40
tau_F_VRA = 20

q_S_BS = 0.1 # 特异收益率-贝叶斯收缩 参数
lag_S_Vol = 5
# 周度调整为20，月度为80和40
tau_S_Vol = 40
tau_S_VRA = 20

FR = FactorRtn.values

BF_F_all=[]
BF_S_all=[]


需要用到的函数

In [146]:
def uniformize(x):
    return x / np.nansum(x)

def normalize(x, weight, std_weight=None):
    mean = np.nansum(x * uniformize(weight))
    if std_weight is not None:
        std = np.nanmean((x-mean)**2 * std_weight)
    else:
        std = np.nanstd(x)
    return (x - mean) / std

def winsorize(x):
    '''确保x是经过标准化的数据'''
    s_pos = max(0,min(1,0.5/(max(x)-3)))
    s_neg = max(0,min(1,0.5/(-min(x)-3)))
    x[x > 3] = 3*(1 - s_pos) + x[x > 3] * s_pos
    x[x < -3] = -3*(1 - s_neg) + x[x < -3] * s_neg
    return x

def winsorize_percentile(x, per_upper=1, per_lower=0):
    upper = np.percentile(x, per_upper*100)
    lower = np.percentile(x, per_lower*100)
    x[x < lower] = lower
    x[x > upper] = upper
    return x

def orthogonalize(raw, ref, weight=None):
    if weight is not None:
        ortho = raw - (raw * ref * weight).sum() / (ref**2 * weight).sum() * ref
    else:
        ortho = raw - np.nansum(raw * ref) / np.nansum(ref**2) * ref
    return ortho

def fillna_with_regression(x, mask, ref):
    x_valid = x[mask,None]
    ref = np.vstack([np.ones_like(ref),ref]).T
    ref_valid = ref[mask,:]
    beta = np.linalg.inv(ref_valid.T @ ref_valid) @ ref_valid.T @ x_valid
    x[~mask] = (ref[~mask,:] @ beta).ravel()
    return x


def data_clean(x, w, fillna_ref=None):
    ValidMask = ~(np.isnan(x) | np.isinf(x))
    x_valid = x[ValidMask]
    w_valid = uniformize(w[ValidMask])
    x_valid = normalize(winsorize(normalize(x_valid, w_valid)),w_valid)
    x[ValidMask] = x_valid
    if fillna_ref is not None:
        x = normalize(fillna_with_regression(x, ValidMask, fillna_ref), w)
    return x

def get_EWMA(length=252, tau=90, norm=1):
    lambd = 0.5**(1./tau)
    w = np.array([lambd**n for n in range(length)][::-1])
    if norm:
        return uniformize(w)
    else:
        return w

def v_parabolicfit_scaling(v_raw, amp=1.6, k_start_fitting=15):
    '''
    1、Put v_raw into parabolic fit, get v_p
    2、Scale the fitted value in proportion to their deivation from 1, get v_s
    '''
    K = len(v_raw)
    y = v_raw[k_start_fitting:]
    x = np.array(range(k_start_fitting,K))
    polyfunc = np.poly1d(np.polyfit(x, y, 2))
    v_p = np.array([polyfunc(xi) for xi in range(k_start_fitting)] + list(y))
    v_s = amp * (v_p - 1) + 1
    return v_s

def getNeweyWestCov(rtn_data, q, tau, length, n_this, is_NW=1):
    '''
    Input:
        rtn_data            The factor / specific return
        q                   Num of periods used to calculate Newey-West Adjusted Covariance
        tau                 The length of Halflife Decay
        length              How many frames are used to calculate the covariance matrix
                            The length must be greater than K (the No. of factors)
        n_this              'n_this-length' to 'n_this' frames are used to get the covariance
        n_forward           The period ahead to calculate bias B; standard deviation of r_ahead / risk_predicted
        is_NW               whether use Newey-West Method

     n_this-length           n_this                 n_this+n_forward
          |---------------------|-------------------------|
                   F                       Bias

    Output:
        F           if is_NW==True: Newey-West Seriel Correlation Adjustment Covariance;
                    elif: Simple Covariance.
    '''
    q=2
    tau=90
    if n_this < length:
        print('ERROR: n_this should not be less than length')
        return
    elif n_this > rtn_data.shape[0]:
        print('ERROR: n_this + n_forward should not be greater than No. of total frames')
        return
    f_raw = rtn_data[n_this-length:n_this,:]

    # calculate Newey-West covariance
    if is_NW:
        # 半衰期加权所用权重，wighted average of factors
        w = get_EWMA(length, tau, 1)
        ewma = np.dot(w[::-1], f_raw)
        f = (f_raw - ewma).T
        Delta_0 = f @ np.diag(w) @ f.T
        # Calculate the cov matrix
        F = Delta_0
        for d in range(1, q+1):
            # 计算滞后d期的协方差（非对称）
            Delta_i = f[:,:-d] @ np.diag(w[d:]) @ f[:,d:].T / np.sum(w[d:])
            # 乘以Bartlett权重系数
            F += ( (1 - d/(q+1.)) * (Delta_i + Delta_i.T) )
    else:
        F = np.cov(f_raw.T)
    # 日频协方差月频化,21 for monthly
    F = 21. * F
    # decomp of NW covariance, Prepare for Eigenfactor Risk Adjustment
    D_diag, U = linalg.eigh(F) # D  in fact the diagonal element of matrix D, a.k.a EigenVar
    # r = (data.iloc[n_start:n_start+n_forward,:]+1).cumprod().iloc[-1,:]-1
    # R_eigen = np.dot(U.T,r)

    # check if U and D_diag is the right eigh-decomposition of F
    if not np.allclose(F,  U @ np.diag(D_diag) @ U.T ):
        print('ERROR in eigh')
        return

    return F, U, D_diag

def getNeweyWestVol(rtn_data, q, tau, length, n_this, is_NW=1):
    '''
    单变量版本的Newey-West调整
    rtn_data 此处为pd.Seires格式 便于使用apply方法计算。
    '''
    f_raw = rtn_data.iloc[n_this-length:n_this]
    if f_raw.dropna().size < length//4:
        ### 数据过少，直接给出波动率为0
        Vol = 0
        return Vol
    f_raw = f_raw.dropna().values
    length = f_raw.size
    if is_NW:
        # 半衰期加权所用权重，wighted average of factors
        w = get_EWMA(length, tau, 1)
        ewma = np.dot(w[::-1], f_raw)
        f = f_raw - ewma
        Var_0 = (f**2 * w).sum()
        # Calculate the cov matrix
        Var = Var_0
        for d in range(1, q+1):
            # 计算滞后d期的协方差（非对称）
            Var_i = (f[:-d] * f[d:] * w[d:]).sum() / np.sum(w[d:])
            # 乘以Bartlett权重系数
            Var += ( (1 - d/(q+1.)) * (Var_i * 2) )
        Vol = np.sqrt(Var)
    else:
        Vol = np.std(f_raw)
    return Vol

def getEigenfactorRiskAdjustCov(F_0, U_0, D_diag, tau, length, M=100, is_NW=1): # M should be 10^4 as in report
    '''
    Input:
        F_0         Un-adjusted factor covariance matrix
        U_0         KxK rotation matrix whose columns are given by the eigenvectors of F_0.
                    U_0[j,k] gives the weight of pure factor j in eigenfactor k .
        D_diag      the diagonal element of D, which is the predicted variances of the eigenfactors
        M           the total number of simulations
        is_NW       whether use Newey-West Method
    Output:
        F_true      Predicted Factor Covariance After Eigenfactor Risk Adjustment
    '''
    K = F_0.shape[0]
    simulated_volatility_biases = np.zeros((M, K))
    for i in range(M):
        # if i%50==0:print(i)
        b_m = np.array([np.random.normal(0, std, length) for std in np.sqrt(D_diag)]) # generate simulated factor returns
        f_m = (U_0 @ b_m).T
        F_m, U_m, D_diag_m = getNeweyWestCov(f_m, q=2, tau=tau, length=length, n_this=length, is_NW=is_NW)
        D_diag_m_true = np.diagonal(U_m.T @ F_0 @ (U_m))
        simulated_volatility_biases[i,:] = D_diag_m_true / D_diag_m
    # Simulated Volatility Biases
    v_raw = np.sqrt(simulated_volatility_biases.mean(axis=0))
    # After Parabolic fit & Scaling
    v_s = v_parabolicfit_scaling(v_raw, amp=1.6, k_start_fitting=16)
    D_true = np.diag(v_s**2 * D_diag)
    F_true = U_0 @ D_true @ U_0.T
    return F_true
    

def getVolatilityRegimeAdjustment(rtn_data, Vol, tau, length, n_this, BF_all):
    '''
    Input:
        rtn_data        The factor / specific return
        Vol             The volatility of factor / specific return estimated on day n_this
        tau             Halflife parameter of VRA
        length          The length of factor_data used to calculate the volatility multiplier
        n_this
        BF_all          Store each day's Factor Cross-Sectional Bias statistic B^F_t
    Output:
        Vol_VRA         the Volatility Regime Adjustment forecast of Factor Volatility
        BF_all          the same
    '''
    tau=42
    w = get_EWMA(length, tau, 1)
    #Vol=np.diag(Vol) # to solve the dimension bug on b
    isValidVol = (Vol != 0)
    ## b is the cross-section standardized factor return
    ## BF is the bias statistic for realized standard deviation of b
    #b = rtn_data[n_this,:][isValidVol] / Vol[isValidVol]
    b = rtn_data[n_this,:][isValidVol]/ Vol[isValidVol]
    BF = np.sqrt(np.mean(b @ b))
    BF_all.append(BF)
    if n_this > (length+243): # deal with NaN in too early data
        # lambda_F is volatility multiplier 
        lambda_F = np.sqrt( np.power(BF_all[-length:],2) @ w )
        Vol_VRA = lambda_F**2 * Vol # lambda_F可以直接平方吗？a number?
    else:
        Vol_VRA = np.array([np.NaN] * len(Vol))
    
    return Vol_VRA, BF_all

def getSVDAdjustment(cov):
    if np.isnan(Cov_F).all():
        return None
    else:
        U,s,V = linalg.svd(cov)
        s[s <= 0] = min(s[s > 0]) / 10
        cov_adj = U @ np.diag(s) @ V
        return cov_adj

def getBlendWeight(Rtn_seires, length, n_this):
    f_raw = Rtn_seires.iloc[n_this-length:n_this].values
    sigma_u = (np.percentile(f_raw, 75) - np.percentile(f_raw, 25)) / 1.35
    f_raw[f_raw < -10*sigma_u] = -10*sigma_u
    f_raw[f_raw > 10*sigma_u] = 10*sigma_u
    sigma_eq = np.std(f_raw)
    Z_u = np.abs((sigma_eq-sigma_u)/sigma_u)
    gamma = min(1, max(0, np.exp(1-Z_u)))
    return gamma

def getBlendedSpecificVol(FactorExposure, Gamma, Vol_TS):
    mask_gamma = (Gamma == 1)
    y = np.log(Vol_TS[mask_gamma])
    x = FactorExposure[mask_gamma,:]
    b = linalg.inv(x.T @ x) @ x.T @ y
    Vol_STR_0 = np.exp(FactorExposure @ b)
    Vol_hat = Gamma * Vol_TS + (1 - Gamma) * np.mean(Vol_TS / Vol_STR_0) * Vol_STR_0
    return Vol_hat

def getBayesianShrinkageResult(Vol, MktCap, n_decile, q):
    '''
    Input:
        Vol             Volaitlity of specific return of each stock
        MktCap          Market Value of each stock
        n_decile        number of group the stocks will be split into by market value
        q               Shrinkage Parameter
    Output:
        Vol_BS          Volatility after Bayesian Shrinkage
    '''
    isValidVol = (Vol != 0)
    Vol_BS = np.zeros(Vol.size)
    Vol = Vol[isValidVol]
    MktCap = MktCap[isValidVol]
    IdxRankedGrouped = np.array_split(np.argsort(MktCap), n_decile) # 从小到大排列的市值对应序号
    # Get the Shrinkage Target of each decile group
    VolTarget_G = np.array([np.average(Vol[idxs], weights=uniformize(MktCap[idxs])) for idxs in IdxRankedGrouped])
    # Get the standard deviation of specific risk forecasts of each decile group
    VolDelta_G = np.array([((Vol[idxs] - vt)**2).sum() / len(idxs) for vt, idxs in zip(VolTarget_G, IdxRankedGrouped)])
    # Broadcast group values to all valid vol
    VolTarget = np.zeros(Vol.size)
    VolDelta = np.zeros(Vol.size)
    for i_group, idxs in enumerate(IdxRankedGrouped):
        VolTarget[idxs] = VolTarget_G[i_group]
        VolDelta[idxs] = VolDelta_G[i_group]
    # v is the shrinkage intensity
    v = 1 / (VolDelta / q * np.abs(Vol - VolTarget) + 1)
    Vol_BS_Valid = v * VolTarget + (1-v) * Vol
    Vol_BS[isValidVol] = Vol_BS_Valid
    return Vol_BS

In [154]:
Vol_S_df = pd.DataFrame(index=TradeDates, columns=SpecificRtn.columns, dtype=float)
# 滚动计算，跑起来特别慢，从03-23跑了一整天
for idx_date in tqdm(range(length,T)):
    date = TradeDates[idx_date]
    StocksValid = SpecificRtn.loc[date,:].dropna().index.intersection(MarketValue.loc[date,:].dropna().index).to_list()
    # print(idx_date, date)
    FactorExposure = pd.read_pickle("fac_data/fac_data" + str(date) + ".pkl")
    StocksValid = [i for i in StocksValid if i in FactorExposure.index]
    
    FactorExposure = FactorExposure.loc[StocksValid,FactorsValid[:-K_inds]].values
    SR = SpecificRtn.loc[:,StocksValid].values
    thisMV = MarketValue.loc[date,StocksValid].values


    # 计算因子收益率的相关系数矩阵
    Cov_F_NW_toCorr, U_toCorr, D_diag_toCorr = getNeweyWestCov(FR, q=lag_F_Corr, tau=tau_F_Corr, length=length, n_this=idx_date, is_NW=1)
    Cov_F = getEigenfactorRiskAdjustCov(Cov_F_NW_toCorr, U_toCorr, D_diag_toCorr, tau=tau_F_Corr, length=length, M=500, is_NW=0)
    Cov_F_tilde = getSVDAdjustment(Cov_F) # 正定化
    
    # 计算特异收益率的波动率
    SpecificRtnValid = SpecificRtn.loc[:,StocksValid]
    Vol_S_NW = SpecificRtnValid.apply(lambda x:getNeweyWestVol(x, q=lag_S_Vol, tau=tau_S_Vol, length=length, n_this=idx_date, is_NW=1),axis=0).values
    Gamma = SpecificRtnValid.apply(lambda x:getBlendWeight(x, length=length, n_this=idx_date), axis=0).values
    Vol_S_Blend = getBlendedSpecificVol(FactorExposure, Gamma, Vol_S_NW)
    Vol_S_BS = getBayesianShrinkageResult(Vol_S_Blend, thisMV, n_decile=10, q=q_S_BS)
    Vol_S_VRA, BF_S_all = getVolatilityRegimeAdjustment(SR, Vol_S_BS, tau=tau_S_VRA, length=length, n_this=idx_date, BF_all=BF_S_all)
    # 记录数据
    if Cov_F_tilde is not None:
        Vol_S_df.loc[date, StocksValid] = Vol_S_VRA
        pd.DataFrame(Cov_F_tilde, index=FactorsValid, columns=FactorsValid).to_pickle(f'FactorCov&SpecificVol/FactorCov_{date}.pkl')
        #

Vol_S_df = Vol_S_df.dropna(how='all',axis=0)
Vol_S_df.to_pickle('FactorCov&SpecificVol/SpecificVol.pkl')



  0%|          | 0/4645 [00:00<?, ?it/s]

现在F,Δ都有了。进一步加工搞成股票协方差矩阵

In [156]:
F = pd.read_pickle("FactorCov&SpecificVol/FactorCov_20040114.pkl")
Delta = pd.read_pickle("FactorCov&SpecificVol/SpecificVol.pkl")

In [158]:
datei = Delta.index[0]
print(datei)
F = pd.read_pickle("FactorCov&SpecificVol/FactorCov_" + str(datei) + ".pkl")
X = pd.read_pickle("fac_data/fac_data" + str(datei) + ".pkl")
X = X.loc[:,F.index]
common_stocks = [i for i in Delta.columns if i in X.index]

# 股票保持一致
X = X.loc[common_stocks]
Delta_i = Delta[common_stocks]

# delta取指定这一天的, 然后变成一个对角矩阵
Delta_i = Delta_i.loc[str(datei)]
Delta_diag = pd.DataFrame(np.diag(Delta_i))
Delta_diag.columns = X.index 
Delta_diag.index = X.index 

CovStocks = X @ F @ X.T + Delta_diag
# CovStocks.to_parquet("StockCov/StockCov_" + str(datei) + ".parquet")

20050117


In [ ]:
# Delta = pd.read_pickle("FactorCov&SpecificVol/SpecificVol.pkl")
# Delta

# for i in range(860,len(weeklastday)):
#     datei = str(weeklastday[i])
#     print(datei)
#     F = pd.read_pickle("FactorCov&SpecificVol/FactorCov_" + str(datei) + ".pkl")

#     X = pd.read_pickle("fac_data/fac_data" + str(datei) + ".pkl")
#     X = X.loc[:,F.index]

#     common_stocks = [i for i in Delta.columns if i in X.index]

#     # 股票保持一致
#     X = X.loc[common_stocks]
#     Delta_i = Delta[common_stocks]

#     # delta取指定这一天的, 然后变成一个对角矩阵
#     Delta_i = Delta_i.loc[str(datei)]
#     Delta_diag = pd.DataFrame(np.diag(Delta_i))
#     Delta_diag.columns = X.index
#     Delta_diag.index = X.index

#     CovStocks = X @ F @ X.T + Delta_diag
#     CovStocks.to_parquet("StockCov_WeekLast/StockCov_" + str(datei) + ".parquet")

20170203
20170210
20170217
20170224
20170303
20170310
20170317
20170324
20170331
20170407
20170414
20170421
20170428
20170505
20170512
20170519
20170526
20170602
20170609
20170616
20170623
20170630
20170707
20170714
20170721
20170728
20170804
20170811
20170818
20170825
20170901
20170908
20170915
20170922
20170929
20171013
20171020
20171027
20171103
20171110
20171117
20171124
20171201
20171208
20171215
20171222
20171229
20180105
20180112
20180119
20180126
20180202
20180209
20180214
20180223
20180302
20180309
20180316
20180323
20180330
20180404
20180413
20180420
20180427
20180504
20180511
20180518
20180525
20180601
20180608
20180615
20180622
20180629
20180706
20180713
20180720
20180727
20180803
20180810
20180817
20180824
20180831
20180907
20180914
20180921
20180928
20181012
20181019
20181026
20181102
20181109
20181116
20181123
20181130
20181207
20181214
20181221
20181228
20190104
20190111
20190118
20190125
20190201
20190215
20190222
20190301
20190308
20190315
20190322
20190329
20190404
2

FileNotFoundError: [Errno 2] No such file or directory: 'FactorCov&SpecificVol/FactorCov_20230303.pkl'

In [ ]:
weeklastday = pd.read_csv("weeklastday.csv", index_col=0)
weeklastday = weeklastday['0'].tolist()

weekfirstday = pd.read_csv("weekfirstday.csv", index_col=0)
weekfirstday = weekfirstday['0'].tolist()

In [ ]:
weeklastday.index(20221125)

1157

In [ ]:
import shutil
for i in range(1000,len(weeklastday)):

    print(weeklastday[i])
    shutil.copyfile("StockCov/StockCov_" + str(weeklastday[i]) + ".parquet", "StockCov_WeekLast/StockCov_" + str(weeklastday[i]) + ".parquet")



20191101


FileNotFoundError: [Errno 2] No such file or directory: 'StockCov/StockCov_20191101.parquet'